# Tic Tac Toe using Quantum Circuits

----------------------------------
By Artash Nath

Twitter @wonrobot

Co-Founder, HotPopRobot.com

Founder, MonitorMyLockdown.com

----------------------------------


## Importing Libraries

In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

# Numpy for data proccesing
import numpy as np

from time import sleep 

## Setting up the Board

In [2]:
# The Tic Tac Toe Board
theBoard = {1: ' ' , 2: ' ' , 3: ' ' ,
            4: ' ' , 5: ' ' , 6: ' ' ,
            7: ' ' , 8: ' ' , 9: ' ' }

# Function for displaying the boarrd
def printBoard(board):
    print(board[1] + '|' + board[2] + '|' + board[3])
    print('-+-+-')
    print(board[4] + '|' + board[5] + '|' + board[6])
    print('-+-+-')
    print(board[7] + '|' + board[8] + '|' + board[9])

## Creating the Quantum "Tic Tac Toe" Solver Circuit 

-------------

Now that I have initialized the tic tac toe board, I needed to come up with a way to use Quantum Circuits to predict the best move to make based on a given board configuration dictionary (board)

---------------

Before anything else, the first thing I needed to do was express the Tic Tac Toe board using Qbits. I used Qiskit to create a 9 Qbit circuit. Each Qbit would represent a different position on the board.

Next, I needed to apply Quantum Gates to the circuit in such a way that when measuring the Qbits, the Qbit representing the optimal place to choose in the game would have the highest probability.

I decided to turn to T Gates to code the rules of winning tic tac toe into my Circuit. T-Gates amplify the probability of a Qbit being measured as 1. The more T-Gates, the higher the probability of measuring 1. So based on how important a rule was, I could assign each Qbit a different number of T-Gates.

-------------------

In my function, there are 4 steps:
    
1. Apply a Hadamard Gate to every qubit representing a possible move on the board (No Gates for positions on the board already taken). This put's all the posible moves in superposition, and forces the probability of all non-possible moves to 0



2. Apply 1 T-Gate to every Qubit representing a position on the board with 1 other "Xs or Os" in the same row or column (complete 2 in a row, or block). This is to encourage it to start to complete 3 in a row if there is no other better move.



3. Apply 2 T-Gates to every Qubit representing a position on the board with 2 other "Xs or Os" in the same row or clolumn (blocking or winning). This amplifies the probability of choosing a position where it's either completing a row/column and winning, or stoping the other playing from completing one.


4. Apply 3 T-Gates to every qubit representing a position with 2 other "Xs or Os" on the same diagonal. (blocking or winning). Completing a diagonal is the strongest move in tic tac toe, so it is assigned the highest number of T-Gates


-------------

Once the final circuit was created, The Quantum Circuit would be run for 512 shots on the IBM Quantum Simulator. The Qbit that returned the value "1" the most number of times would be the most optimal move to make.

-------------

This process would be repeated for each time the Tic-Tac-Toe board was updated. i.e: a new circuit created for every move

In [3]:
# Given a board configuration, this function creates a quantum circuit and repeats the above steps to predict the best move
def botturn(board):
    
    num_qubits = 9
    # reset the T gate counter
    num_t_gates = [0] * num_qubits
    
    # Base Circuit
    q = QuantumRegister(num_qubits)
    c = ClassicalRegister(num_qubits)
    qc = QuantumCircuit(q, c)
    
    for i in range(1,10):
        
        cspace = board[i]
        index = i-1
        
        if cspace!=' ':
            pass
            
        else:
            qc.h(index)
            
            t_count = 0
            
#==================================================================================================
# COLUMN POSITION RULES
#==================================================================================================
            if (index) % 3 == 0: # Start of a Horizontal Side
                if ((board[i+1]) == (board[i+2])) and (board[i+1]!=' '): # Block / Win (if other 2 in row are occupied)
                    qc.t(index)
                    qc.t(index)
                    t_count+=2
                elif (board[i+1]!=' ') or (board[i+2]!=' '): # if one another in that row is occupied
                    qc.t(index)
                    t_count+=1
                    
            
                    
                    
            if (index) % 3 == 1: # Middle of a Row
                if ((board[i-1]) == (board[i+1])) and (board[i+1]!=' '): # Block / Win (if other 2 in row are occupied)
                    qc.t(index)
                    qc.t(index)
                    t_count+=2
                elif (board[i-1]!=' ') or (board[i+1]!=' '): # if one another in that row is occupied
                    qc.t(index)
                    t_count+=1
                    
                    
            if (index) % 3 == 2: # End of a Row
                if ((board[i-1]) == (board[i-1])) and (board[i-1]!=' '): # Block / Win (if other 2 in row are occupied)
                    qc.t(index)
                    qc.t(index)
                    t_count+=2
                elif (board[i-1]!=' ') or (board[i-2]!=' '): # if one another in that row is occupied
                    qc.t(index)
                    t_count+=1
                    
            
                                    
#==================================================================================================      
# ROW POSITION RULES
#==================================================================================================               

                    
            if index <=2: # Top Row
                if (board[i+3] == board[i+6]) and (board[i+3]!=' '): # If both in column are occupied
                    qc.t(index)
                    qc.t(index)
                    t_count+=2         
                elif (board[i+3]!=' ') or (board[i+6]!=' '): # if one another in that column is occupied
                    qc.t(index)
                    t_count+=1     
                    
                    
            if 3 <= index <= 5: # Middle row
                if (board[i-3] == board[i+3]) and (board[i+3]!=' '): # If both in column are occupied
                    qc.t(index)
                    qc.t(index)
                    t_count+=2         
                elif (board[i-3]!=' ') or (board[i+3]!=' '): # if one another in that column is occupied
                    qc.t(index)
                    t_count+=1   
            
            
            if 6 <= index <= 8: # Bottom row
                if (board[i-3] == board[i-6]) and (board[i-3]!=' '): # If both in column are occupied
                    qc.t(index)
                    qc.t(index)
                    t_count+=2         
                elif (board[i-3]!=' ') or (board[i-6]!=' '): # if one another in that column is occupied
                    qc.t(index)
                    t_count+=1   
                    
            num_t_gates[index] = t_count
           
    
#==================================================================================================      
# DIAGONAL POSITION RULES
#================================================================================================== 

    if (board[1] == board[5]) and (board[1]!=' '):
        qc.t(8)
        qc.t(8)
        qc.t(8)
        num_t_gates[8] += 3
    if (board[1] == board[9]) and (board[1]!=' '):
        qc.t(4)
        qc.t(4)
        qc.t(4)
        num_t_gates[4] += 3
    if (board[5] == board[9]) and (board[5]!=' '):
        qc.t(0)
        qc.t(0)
        qc.t(0)
        num_t_gates[0] += 3
    if (board[3] == board[5]) and (board[5]!=' '):
        qc.t(6)
        qc.t(6)
        qc.t(6)
        num_t_gates[6] += 3
    if (board[3] == board[7]) and (board[7]!=' '):
        qc.t(4)
        qc.t(4)
        qc.t(4)
        num_t_gates[4] += 3
    if (board[5] == board[7]) and (board[7]!=' '):
        qc.t(2)
        qc.t(2)
        qc.t(2)
        num_t_gates[2] += 3
#==================================================================================================               

    for i in range(1,10):
        cspace = board[i]
        index = i-1
        
        if cspace==' ':
            qc.h(index)
        else:
            num_t_gates[index]-=1
            
    qc.measure(q, c)
    backend = Aer.get_backend('qasm_simulator')
    shots = 512
    job_sim = execute(qc, backend, shots=shots)
    sim_result = job_sim.result().get_counts(qc)

    counts = [0]*num_qubits
    
    for key, count in sim_result.items():
        # need to iterate over the results and see when the value was chosen most
        # keys are the opposite way round to expected
        key = key[::-1]
        for index, val in enumerate(key):
            if val == '1':
                counts[index] += count


    move = np.argmax(np.array(list(enumerate(counts)))[:,1])    
    qc.draw(output='mpl')
    #print(qc)
    #print(num_t_gates)
    return move

# The Game

In [4]:
def Game():
    
    theBoard = {1: ' ' , 2: ' ' , 3: ' ' ,
                4: ' ' , 5: ' ' , 6: ' ' ,
                7: ' ' , 8: ' ' , 9: ' ' }
    
    while True:
        
        count = 0
        
        printBoard(theBoard)
        print("It's your turn, Move to which place?")
        
        move = int(input())   
        
        print(theBoard[move])
        if (type(move)==int)  and (move<=9)  and (move>=1) and (theBoard[move] == ' '):
                theBoard[move] = 'X'
                count += 1
        else:
            print("Invalid Move")
            continue
           
        print('You have played your turn:')
        printBoard(theBoard)
        
        
        botmove = botturn(theBoard)
        theBoard[botmove+1]='O'
        sleep(1.2)
        print('QuantumBot has Executed its turn')
        print()
        
        

In [ ]:
Game() # To play

 | | 
-+-+-
 | | 
-+-+-
 | | 
It's your turn, Move to which place?
